In [14]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
from sklearn.preprocessing import scale
from sklearn.model_selection import cross_val_score
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from PIL import Image
from sklearn.metrics import accuracy_score

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import AnovaRM

from matplotlib import pyplot as plt
from matplotlib import pyplot
import seaborn as sns
import xgboost as xgb
from sklearn.naive_bayes import GaussianNB

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from xgboost import XGBClassifier
from sklearn.feature_selection import RFE


In [15]:
db = pd.read_csv("data.csv",sep = ';' )
db.head()

,ATES,BULANTI,BEL-AGRI,SUREKLI-WC,IDRAR-SIRASINDA-AGRI,URETRADA-YANMA-SISME-KASINTI,MESANE-ILTIHABI,BOBREK-ILTIHABI
0,355,0,1,0,0,0,0,0
1,359,0,0,1,1,1,1,0
2,359,0,1,0,0,0,0,0
3,360,0,0,1,1,1,1,0
4,360,0,1,0,0,0,0,0


In [16]:
dt = DecisionTreeClassifier()
rf = RandomForestClassifier(n_estimators=100, random_state=0, n_jobs = -1)
logreg = LogisticRegression(solver='liblinear',multi_class='ovr')
svm = SVC(gamma='auto')
xgb = xgb.XGBClassifier()
gnb = GaussianNB()
models = [dt, rf, logreg, svm, xgb, gnb]

In [4]:
db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 8 columns):
 #   Column                        Non-Null Count  Dtype
---  ------                        --------------  -----
 0   ATES                          120 non-null    int64
 1   BULANTI                       120 non-null    int64
 2   BEL-AGRI                      120 non-null    int64
 3   SUREKLI-WC                    120 non-null    int64
 4   IDRAR-SIRASINDA-AGRI          120 non-null    int64
 5   URETRADA-YANMA-SISME-KASINTI  120 non-null    int64
 6   MESANE-ILTIHABI               120 non-null    int64
 7   BOBREK-ILTIHABI               120 non-null    int64
dtypes: int64(8)
memory usage: 7.6 KB


In [5]:
X = db.drop(["MESANE-ILTIHABI","BOBREK-ILTIHABI"],axis=1)
y = db['MESANE-ILTIHABI']
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2 )

In [6]:
A = db.drop(["MESANE-ILTIHABI","BOBREK-ILTIHABI"],axis=1)
b = db['BOBREK-ILTIHABI']
A_train, A_test, b_train, b_test = train_test_split(A, b,test_size=0.2 )

In [7]:
def model(name, f_train, f_test, d_train, d_test):
    name.fit(f_train, d_train)
    y_pred = name.predict(f_test)
    score= accuracy_score(d_test, y_pred) * 100
    print(str(name) + "Accuracy:",score)
    report_name=classification_report(d_test, y_pred)
    print(report_name)
    print(confusion_matrix(d_test,y_pred))
def rfe(model, features, decision):  
    rfe = RFE(model, 2)
    fit = rfe.fit(features, decision)
    print("Num Features: %s" % (fit.n_features_))
    print("Selected Features: %s" % (fit.support_))
    print("Feature Ranking: %s" % (fit.ranking_))
def crossval(model, features, decision , n):
    scores=cross_val_score(model, features, decision, cv=n , scoring ="accuracy")
    print(str(model) + "Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [8]:
for i in models:
    model(i, X_train, X_test, y_train, y_test)
    model(i, A_train, A_test, b_train, b_test)
    crossval(i, X, y, 5)
    crossval(i, X, y, 10)
    crossval(i, A, b, 5)
    crossval(i, A, b, 10)
    

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')Accuracy: 100.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        12
           1       1.00      1.00      1.00        12

    accuracy                           1.00        24
   macro avg       1.00      1.00      1.00        24
weighted avg       1.00      1.00      1.00        24

[[12  0]
 [ 0 12]]
DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impu

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='ovr', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)Accuracy: 1.00 (+/- 0.00)
SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)Accuracy: 83.33333333333334
              precision    recall  f1-score   support

           0       0.83      0.83      0.83        12
           1       0.83      0.83      0.83        12

    accuracy                           0.83        24
   macro avg       0.83      0.83      0.83        24
weighted avg       0.83      0.83      0.83        24

[[10  2]
 [ 2 10]]
SVC(C=1.0, break_ties=Fa

In [9]:
model = [dt, rf, logreg, xgb]
for i in model:
    print(i)
    rfe(i, X, y)
    rfe(i, A, b)


DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')
Num Features: 2
Selected Features: [False False False  True  True False]
Feature Ranking: [4 5 2 1 1 3]
Num Features: 2
Selected Features: [ True False  True False False False]
Feature Ranking: [1 5 1 4 3 2]
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
             

In [10]:
new_input = [[350, 0, 1, 1, 1, 0]]

In [23]:
def prob(model, new_input):
    m1 = model
    m1.fit(X_train, y_train)
    d1 = m1.predict(new_input)
    prb1 = m1.predict_proba(new_input)
    print(d1, prb1)
    m2 = model
    m2.fit(A_train, b_train)
    d2 = m2.predict(new_input)
    prb2 = m2.predict_proba(new_input)
    print(d2, prb2)
    if d1==0 or d1==1:
        print("mesane iltihabı olma oranınız :"  ,prb1[0,1])
    if d2==0 or d2==1:
        print("böbrek iltihabı olma oranınız :" ,prb2[0,1] )
    

In [24]:
prob(logreg, new_input)

[1] [[0.22720319 0.77279681]]
[0] [[0.50456243 0.49543757]]
mesane iltihabı olma oranınız : 0.7727968127159675
böbrek iltihabı olma oranınız : 0.49543756846442444
